# EVALUATION NOTEBOOK for TACAS 2025 PAPER

# Pattern & Triggers
Receieve data from a run with ACTIONTYPE=NOTHING but NEW_TRIGGER=true

In [23]:
import helper_methods
import load_data
import group_methods
import result_analysis
import numpy as np
import validate
import pandas as pd
import patterns_triggers

In [24]:
df = load_data.load_data_mult('analysis_csv')
limited_df = helper_methods.get_df_rows_no_timeout(df)[['Spec','UNIQUE_PATTERNS','PATTERN_INSTANCES','TRIGGER_INSTANCES']].drop_duplicates().reset_index(drop=True)
df_len = len(limited_df)
print(f"amount of rows:{df_len}")
limited_df

Found 2 CSV file(s) in the 'analysis_csv' directory.
Loaded: all_runs_new_SPLIT1_ws1_20241009_235902.csv
Number of specs in this file: 80
Loaded: all_runs_new_SPLIT2_ws2_20241009_221236.csv
Number of specs in this file: 98
Total number of files loaded: 2
Total Specs: 178
Final DataFrame shape: (1068, 88)
amount of rows:178


,Spec,UNIQUE_PATTERNS,PATTERN_INSTANCES,TRIGGER_INSTANCES
0,1670057023,S_responds_to_P_globally,3,1
1,1669965418,S_responds_to_P_globally,2,0
2,1669902814,S_responds_to_P_globally,2,0
3,RobotsMPUnlabelled3,NaN,0,0
4,1669665995,NaN,0,0
...,...,...,...,...
173,1670436736,NaN,0,3
174,1670542951,P_is_true_After_Q_until_R,3,0
175,1672062051,NaN,0,2
176,1670601332,NaN,0,0


## Pattern Data

In [25]:
# Combine all patterns into a single list
all_patterns = limited_df['UNIQUE_PATTERNS'].str.split(',').explode().dropna()

# Create a dictionary to count the usage of each pattern
pattern_usage = all_patterns.value_counts().to_dict()

# Get unique patterns
unique_patterns = all_patterns.drop_duplicates()

# Count the amount of unique patterns
unique_pattern_count = len(unique_patterns)

# Print the total amount and proportion of unique patterns
total_patterns = len(all_patterns)
print(f"Total patterns used: {total_patterns}")
print(f"Unique patterns: {unique_pattern_count}")
print(f"Proportion of unique patterns: {unique_pattern_count / total_patterns:.2%}")

# Filter specs that don't have patterns
pattern_df = df[df['PATTERN_INSTANCES'] > 0]

# Count the amount of patterns with instances
pattern_count = len(pattern_df)

# Print the proportion of patterns with instances
df_len = len(df)  # Assuming df is defined elsewhere
print(f"Patterns with instances: {pattern_count}")
print(f"Proportion of patterns with instances: {pattern_count / df_len:.2%}")

# Print the usage count for each pattern
print("\nPattern usage counts:")
for pattern, count in pattern_usage.items():
    print(f"{pattern}: {count}")


Total patterns used: 107
Unique patterns: 10
Proportion of unique patterns: 9.35%
Patterns with instances: 588
Proportion of patterns with instances: 55.06%

Pattern usage counts:
S_responds_to_P_globally: 73
P_is_false_before_R: 7
P_becomes_true_between_Q_and_R: 6
P_becomes_true_after_Q_until_R: 6
P_is_true_After_Q_until_R: 4
P_becomes_true_After_Q: 4
P_is_false_after_Q_until_R: 3
P_is_true_between_Q_and_R: 2
P_becomes_true_before_R: 1
P_is_false_between_Q_and_R: 1


In [26]:
# Filter specs that have patterns
def keep_first_if_equal(group):
    if group.nunique() == 1:
        return group.iloc[0]
    else:
        return pd.Series(np.nan, index=group.index)

no_duplicates_df = df[['Spec','PATTERN_INSTANCES','UNIQUE_PATTERNS']].groupby('Spec').agg({keep_first_if_equal})
no_duplicates_df


,PATTERN_INSTANCES,UNIQUE_PATTERNS
,keep_first_if_equal,keep_first_if_equal
Spec,,
1669468393,0,"[nan, nan, nan, nan, nan, nan]"
1669640935,2,S_responds_to_P_globally
1669643093,2,S_responds_to_P_globally
1669643605,2,S_responds_to_P_globally
1669665995,0,"[nan, nan, nan, nan, nan, nan]"
...,...,...
variant-2-4,6,"P_becomes_true_after_Q_until_R,P_is_false_afte..."
variant-2-8,6,"P_is_false_after_Q_until_R,S_responds_to_P_glo..."


In [27]:
all_patterns

0            S_responds_to_P_globally
1            S_responds_to_P_globally
2            S_responds_to_P_globally
5            S_responds_to_P_globally
6            S_responds_to_P_globally
                    ...              
164          S_responds_to_P_globally
167    P_becomes_true_after_Q_until_R
170          S_responds_to_P_globally
171          S_responds_to_P_globally
174         P_is_true_After_Q_until_R
Name: UNIQUE_PATTERNS, Length: 107, dtype: object

## Trigger Data

In [28]:
# Filter rows where TRIGGER_INSTANCES > 0
trigger_df = limited_df[limited_df['TRIGGER_INSTANCES'] > 0]

# Count the amount of triggers
trigger_count = len(trigger_df)

# Print the proportion of triggers
print(f" Specs with Triggers: {trigger_count}")
print(f"Proportion of triggers: {trigger_count / len(limited_df):.2%}")

 Specs with Triggers: 55
Proportion of triggers: 30.90%


## Stats for Pattern and Trigger Reduction

In [29]:
df['REDUCED_PATTERN']

0       []
1       []
2       []
3       []
4       []
        ..
1063    []
1064    []
1065    []
1066    []
1067    []
Name: REDUCED_PATTERN, Length: 1068, dtype: object

In [30]:
df_no_timeout = helper_methods.get_df_rows_no_timeout(df)

# Process Patterns
print("Analysis for SPECS WITH PATTERNS:")
df_with_pattern = df_no_timeout[df_no_timeout['PATTERN_INSTANCES'] > 0]
patterns_triggers.get_prop(df_with_pattern, 'REDUCED_PATTERN', ['REMOVED_VAR', 'REMOVED_JUSTICE', 'REMOVED_HOLE'])

print("\nAnalysis for SPECS WITH REDUCIBLE PATTERNS:")
df_with_reducible_patterns = df_no_timeout[df_no_timeout['REDUCED_PATTERN'].apply(lambda x: 'REMOVED_HOLE' in patterns_triggers.string_to_set(x))]
patterns_triggers.get_prop(df_with_reducible_patterns, 'REDUCED_PATTERN', ['REMOVED_VAR', 'REMOVED_JUSTICE', 'REMOVED_HOLE'])

# Process Triggers
df_with_trigger = df_no_timeout[df_no_timeout['TRIGGER_INSTANCES'] > 0]
print("\nAnalysis for SPECS WITH TRIGGERS:")
patterns_triggers.get_prop(df_with_trigger, 'REDUCED_TRIGGER', ['REMOVED_JUSTICE', 'REMOVED_HOLE'])

print("\nAnalysis for SPECS REDUCABLE TRIGGERS:")
df_with_reducible_triggers = df_no_timeout[df_no_timeout['REDUCED_TRIGGER'].apply(lambda x: 'REMOVED_HOLE' in patterns_triggers.string_to_set(x))]
patterns_triggers.get_prop(df_with_reducible_triggers, 'REDUCED_TRIGGER', ['REMOVED_VAR','REMOVED_JUSTICE', 'REMOVED_HOLE'])

Analysis for SPECS WITH PATTERNS:
Proportion of specs with REMOVED_VAR: 9.18% (54/588)
Proportion of specs with REMOVED_JUSTICE: 9.18% (54/588)
Proportion of specs with REMOVED_HOLE: 11.73% (69/588)

Analysis for SPECS WITH REDUCIBLE PATTERNS:
Proportion of specs with REMOVED_VAR: 78.26% (54/69)
Proportion of specs with REMOVED_JUSTICE: 78.26% (54/69)
Proportion of specs with REMOVED_HOLE: 100.00% (69/69)

Analysis for SPECS WITH TRIGGERS:
Proportion of specs with REMOVED_JUSTICE: 8.18% (27/330)
Proportion of specs with REMOVED_HOLE: 50.00% (165/330)

Analysis for SPECS REDUCABLE TRIGGERS:
Proportion of specs with REMOVED_VAR: 12.73% (21/165)
Proportion of specs with REMOVED_JUSTICE: 16.36% (27/165)
Proportion of specs with REMOVED_HOLE: 100.00% (165/165)


### Ratio for JUSTICE

In [31]:
df_grouped = group_methods.group(df)
df_grouped_no_timeout = helper_methods.get_df_specs_no_timeout(df_grouped)

metric_columns = ['WORK_TIME', 'TOTAL_TIME']
runconfigs = df_grouped_no_timeout['RunConfig'].unique()



In [32]:
def get_ratio(df,is_pattern,baseline_config):
    if is_pattern:
        cond= "REDUCED_PATTERN"
    else: 
        cond = "REDUCED_TRIGGER"
    df_just = df[df[cond].apply(lambda x: 'REMOVED_JUSTICE' in patterns_triggers.string_to_set(x))]
    df_var = df[df[cond].apply(lambda x: 'REMOVED_VAR' in patterns_triggers.string_to_set(x))]
    df_var_just = df_just[df_just[cond].apply(lambda x: 'REMOVED_VAR' in patterns_triggers.string_to_set(x))]
    
    print(f"JUSTICE REMOVED")
    pivot_df = result_analysis.pivot(df_just, metric_columns)
    geometric_averages = result_analysis.get_geometric_avg(pivot_df,runconfigs,baseline_config)
    result_analysis.print_headtohead_ratio(geometric_averages,pivot_df.columns,runconfigs)
    
    print(f"VAR REMOVED")
    pivot_df = result_analysis.pivot(df_var, metric_columns)
    geometric_averages = result_analysis.get_geometric_avg(pivot_df,runconfigs,baseline_config)
    result_analysis.print_headtohead_ratio(geometric_averages,pivot_df.columns,runconfigs)
    
    print(f"VAR AND JUSTICE REMOVED")
    pivot_df = result_analysis.pivot(df_var_just, metric_columns)
    geometric_averages = result_analysis.get_geometric_avg(pivot_df,runconfigs,baseline_config)
    result_analysis.print_headtohead_ratio(geometric_averages,pivot_df.columns,runconfigs)


### Ratio for Justice Triggers

## Filter To Get SYNTECH-PATTERNS

In [33]:
from jupyter_server import files


def save_csv(name, csv):
    # Save to a CSV file
    csv_filename = name
    csv.to_csv(csv_filename, index=False)

    # Download the file
    files.download(csv_filename)


## Verify REMOVED VAR

In [34]:
# Assuming patterns_triggers.string_to_set() is a defined function

removed_pattern_var = df['REDUCED_PATTERN'].apply(lambda x: 'REMOVED_VAR' in patterns_triggers.string_to_set(x))
removed_trigger_var = df['REDUCED_TRIGGER'].apply(lambda x: 'REMOVED_VAR' in patterns_triggers.string_to_set(x))

# Verify that REMOVED_VAR actually removed
actual_removed_var = df.groupby('Spec')['TOTAL_DOMS'].agg(lambda x: x.nunique() == 1)

if not np.array_equal(actual_removed_var, (removed_trigger_var | removed_pattern_var | actual_removed_var)):
    raise ValueError("The conditions are not equal")

ValueError: The conditions are not equal

In [ ]:
temp = df[['Spec']].copy()
temp['keep'] = df['REDUCED_PATTERN'].apply(lambda x: 'REMOVED_HOLE' in patterns_triggers.string_to_set(x))
temp = temp.filter(['Spec','keep'])
temp['keep'] =temp['keep'].replace({False:0,True:1})

save_csv('reduced_patterns.csv', temp)


## Filter to Get SYNTECH-TRIGGERS

In [ ]:
temp = df[['Spec']].copy()
temp['keep'] = df['REDUCED_TRIGGER'].apply(lambda x: 'REMOVED_HOLE' in patterns_triggers.string_to_set(x))
temp = temp.filter(['Spec','keep'])
temp['keep'] =temp['keep'].replace({False:0,True:1})

save_csv('reduced_triggers.csv', temp)
